In [ ]:
%scala
val storage_container = "data"
val storage_account_name = "<NAME>"
val storage_account_access_key = "<KEY>"
val storage_base_url = s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net"

val file_type = "csv"
val train_data_url = s"${storage_base_url}/data/train/train_data/*.${file_type}"
val  test_data_url = s"${storage_base_url}/data/test/test_data/*.${file_type}"

storage_container: String = data
storage_account_name: String = cs1100320004e0010e6
storage_account_access_key: String = an5UdS00UuMcb6vT54s2kWCF31rW++P9BKK7K0czqJDmONenltWmfatssB+4tAQOAHbYOUALp2RB+AStHj88Gg==
storage_base_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net
file_type: String = csv
train_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/train/train_data/*.csv
test_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/test/test_data/*.csv

In [ ]:
%scala
import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType, DoubleType, ArrayType}
import org.apache.spark.sql.SparkSession

val spark = ( SparkSession.builder()
                          .appName("BERT")
                          .config(s"fs.azure.account.key.${storage_account_name}.blob.core.windows.net", storage_account_access_key)
                          .getOrCreate() )
val sc = spark.sparkContext

import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType, DoubleType, ArrayType}
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5257b8e
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@6366a42a

In [ ]:
%scala
val stopWordsEN = Array(
 "i","me","my","myself",
 "we","our","ours","ourselves",
 "you","you're","you've","you'll","you'd","your","yours","yourself","yourselves",
 "he","him","his","himself",
 "she","she's","her","hers","herself",
 "it","it's","its","itself",
 "they","them","their","theirs","themselves",
 "what","which","who","whom",
 "this","that","that'll","these","those",
 "am","is","are","was","were","be","been","being",
 "have","has","had","having",
 "do","does","did","doing",
 "a","an","the",
 "of","at","by","for","with",
 "about","against","between","into","through","during",
 "before","after",
 "above","below",
 "to","from","up","down","in","out","on","off","over","under",
 "again","further","then","once","here","there",
 "when","where","why","how","all","any","both","each",
 "s","t",
 "can","will","just",
 "should","should've",
 "now","d","ll","m","o","re","ve","y","ma"
)

stopWordsEN: Array[String] = Array(i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, s, t, can, will, just, should, should've, now, d, ll, m, o, re, ve, y, ma)

In [ ]:
%scala
import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, BertEmbeddings, StopWordsCleaner}
import com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings
import com.johnsnowlabs.nlp.base.DocumentAssembler
import com.johnsnowlabs.nlp.EmbeddingsFinisher
import org.apache.spark.ml.Pipeline
import spark.implicits._

val documentAssembler = new DocumentAssembler()
  .setInputCol("comment")
  .setOutputCol("document")

val documentNormalizer = new DocumentNormalizer()
  .setInputCols("document")
  .setOutputCol("normalizedDocument")
  .setAction("clean")
  .setPatterns(Array("<[^>]>", """[^\w\d\s]"""))
  .setReplacement(" ")
  .setPolicy("pretty_all")
  .setLowercase(true)

val tokenizer = new Tokenizer()
  .setInputCols(Array("normalizedDocument"))
  .setOutputCol("token")

val stopWords = new StopWordsCleaner()
  .setInputCols("token")
  .setOutputCol("cleanTokens")
  .setStopWords(stopWordsEN)

val lemmatizer = LemmatizerModel.pretrained()
  .setInputCols(Array("cleanTokens"))
  .setOutputCol("lemma")

val embeddings = BertEmbeddings.pretrained("bert_embeddings_bert_large_uncased_whole_word_masking")
  .setInputCols("cleanTokens", "normalizedDocument")
  .setOutputCol("bert_embeddings")

val embeddingsSentence = new SentenceEmbeddings()
  .setInputCols(Array("normalizedDocument", "bert_embeddings"))
  .setOutputCol("sentence_embeddings")
  .setPoolingStrategy("AVERAGE")
  .setDimension(768)

val embeddingsFinisher = new EmbeddingsFinisher()
  .setInputCols("sentence_embeddings")
  .setOutputCols("finished_embeddings")
  .setOutputAsVector(false)
  .setCleanAnnotations(true)

val pipeline = new Pipeline()
  .setStages(Array(
    documentAssembler,
    documentNormalizer,
    tokenizer,
    stopWords,
    lemmatizer,
    embeddings,
    embeddingsSentence,
    embeddingsFinisher
  ))

bert_embeddings_bert_large_uncased_whole_word_masking download started this may take some time.
Approximate size to download 1.2 GB
Download done! Loading the resource.
import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, BertEmbeddings, StopWordsCleaner}
import com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings
import com.johnsnowlabs.nlp.base.DocumentAssembler
import com.johnsnowlabs.nlp.EmbeddingsFinisher
import org.apache.spark.ml.Pipeline
import spark.implicits._
documentAssembler: com.johnsnowlabs.nlp.DocumentAssembler = document_362238b44f5a
documentNormalizer: com.johnsnowlabs.nlp.annotators.DocumentNormalizer = DOCUMENT_NORMALIZER_66cfe408e888
tokenizer: com.johnsnowlabs.nlp.annotators.Tokenizer = REGEX_TOKENIZER_32c4944d8b28
stopWords: com.johnsnowlabs.nlp.annotators.StopWordsCleaner = STOPWORDS_CLEANER_300d314960f7
lemmatizer: com.johnsnowlabs.nlp.annotators.LemmatizerModel = LEMMATIZER_c62ad8f355f9
embeddings: com.johnsnowlabs.nlp.embeddings.BertEmbeddings = BERT_EMBEDDINGS_a4578d01508a
embeddingsSentence: com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings = SENTENCE_EMBEDDINGS_81859669c2c9
embeddingsFinisher: com.johnsnowlabs.nlp.EmbeddingsFinisher = embeddings_finisher_5f53e5825fa7
pipeline: org.apache.spark.ml.Pipeline = pipeline_86ac09e095ac

In [ ]:
%scala
val schema = StructType(
  StructField("id"     , IntegerType, true) ::
  StructField("rating" , IntegerType, true) ::
  StructField("type"   , IntegerType, true) ::
  StructField("comment", StringType , true) :: Nil
)

val trainDF = spark.read.format("csv")
                        .schema(schema)
                        .option("header","true")
                        .load(train_data_url)
                        .filter( col("type") =!= -1 )

val testDF  = spark.read.format("csv")
                        .schema(schema)
                        .option("header","true")
                        .load(test_data_url)
                        .filter( col("type") =!= -1 )

val transformedTrainDF = pipeline.fit(trainDF)
                                 .transform(trainDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("finished_embeddings")(0).as("bertEmbeddings")
                                  )
                                 .cache()

val transformedTestDF  = pipeline.fit(testDF)
                                 .transform(testDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("finished_embeddings")(0).as("bertEmbeddings")
                                  )
                                 .cache()

schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,true),StructField(rating,IntegerType,true),StructField(type,IntegerType,true),StructField(comment,StringType,true))
trainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
testDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
transformedTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
transformedTestDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]

In [ ]:
%scala
transformedTrainDF.show(5, 80)

+-----+------+---------+--------------------------------------------------------------------------------+
 id|rating|sentiment| bertEmbeddings|
+-----+------+---------+--------------------------------------------------------------------------------+
 1175| 9| 1|[0.3903178, -0.47742915, 0.25363666, -0.25420463, 0.18092123, -0.029272322, -...|
 4383| 9| 1|[-0.11558779, -0.13197686, 0.0900983, -0.20005278, 0.047314033, 0.021220865, ...|
10044| 9| 1|[0.04631238, -0.42852083, -0.116224274, 0.082751885, 0.29743165, -0.1635426, ...|
 4076| 10| 1|[0.21556105, -0.22938758, 0.110332474, -0.10473291, 0.4642999, 0.11725152, -0...|
 2662| 10| 1|[0.049545724, -0.3437926, 0.17720522, -0.16182216, 0.22136936, 0.04447625, -0...|
+-----+------+---------+--------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTestDF.show(5, 80)

+----+------+---------+--------------------------------------------------------------------------------+
 id|rating|sentiment| bertEmbeddings|
+----+------+---------+--------------------------------------------------------------------------------+
8647| 10| 1|[0.06451508, -0.5604385, 0.19125977, -0.14049812, 0.4217984, -0.014832542, -0...|
3801| 7| 1|[0.041768502, -0.5388923, 0.1286336, -0.027500259, 0.2693935, -0.027567767, -...|
4051| 10| 1|[0.06295091, -0.3892817, -0.12723964, -0.307009, 0.4160712, -0.171942, -0.378...|
 754| 9| 1|[0.035307005, -0.43192902, 0.068607464, -0.19566707, 0.21082292, -0.20589036,...|
3958| 9| 1|[0.11700268, -0.22061978, 0.17566434, -0.26086786, 0.26458237, -0.08206427, -...|
+----+------+---------+--------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTrainDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/BertEmbeddings_train")

transformedTestDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/BertEmbeddings_test")